In [ ]:
import os
import numpy as np
from pathlib import Path
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

from datasets import load_dataset
import pandas as pd

In [ ]:
# Before executing this code download the court order files with 
#
#    https://github.com/niklaswais/gesp 
#
# and specify the download directory below in gesp_dir.
#--------------------------------------------------------------------

gesp_dir = Path('D:\Projects\gesp')

In [ ]:
# get all court order filenames and associated federated state names
#--------------------------------------------------------------------

state_dirs =  [f for f in gesp_dir.iterdir() if f.is_dir()]

for f in state_dirs:
    print(f)

files = [f for f in gesp_dir.iterdir() if f.is_file()]

f = state_dirs[0]

order = [];     # court order file name
order_st = [];  # court order federated state name

states = {'bb':'Brandenburg',
          'be':'Berlin',
          'bw':'Baden-Württemberg',
          'by':'Bayern',
          'he':'Hessen',
          'hh':'Hamburg',
          'mv':'Mecklenburg-Vorpommern',
          'ni':'Niedersachsen',
          'nw':'Nordrhein-Westfalen',
          'rp':'Rheinland-Pfalz',
          'sh':'Schleswig-Holstein',
          'sl':'Saarland',
          'st':'Sachsen-Anhalt',
          'th':'Thüringen'}

for path, subdirs, files in os.walk(gesp_dir):
    for file in files:
        if file.endswith(".html"):
            order.append(os.path.join(path, file))
            order_st.append(path[-2:])


print('%g court orders found.' % (len(order)))

In [ ]:
# create text phrases to be detected in the verdicts
#--------------------------------------------------------------------

tmp_cl0  = ['Kosten haben die PLU1 zu tragen',
            'Kosten hat der MAS1 zu tragen',
            'Kosten hat die FEM1 zu tragen',
            'PLU1 haben die Kosten zu tragen',
            'MAS1 hat die Kosten zu tragen',
            'FEM1 hat die Kosten zu tragen',
            'FEM1 trägt die Kosten',
            'MAS1 trägt die Kosten',
            'PLU1 tragen die Kosten',
            'Kosten trägt der MAS1',
            'Kosten trägt die FEM1',
            'Kosten tragen die PLU1',
            'Kosten werden den PLU2 auferlegt',
            'Kosten werden der FEM2 auferlegt',
            'Kosten werden dem MAS2 auferlegt',      
            'PLU2 werden die Kosten auferlegt',
            'FEM2 werden die Kosten auferlegt',
            'MAS2 werden die Kosten auferlegt']

dict0 = {'MAS1': ['Kläger','Antragsteller','Beschwerdeführer'],
         'FEM1': ['Klägerin','Antragstellerin','klagende Partei','Beschwerdeführerin'],
         'PLU1': ['Kläger','Antragsteller','Beschwerdeführer'],
         'MAS2': ['Kläger','Antragsteller','Beschwerdeführer'],
         'FEM2': ['Klägerin','Antragstellerin','klagenden Partei','Beschwerdeführerin'],
         'PLU2': ['Klägern','Antragstellern','Beschwerdeführern']
        }

text_cl0 = ['abgewiesen',
            'zurückgewiesen']

for t in tmp_cl0:
    for j,(k,v) in enumerate(dict0.items()): 
        if t.find(k)>-1:
            for i,marker in enumerate(v):
                text_cl0 += [t.replace(k,marker)] 


tmp_cl1  = ['Kosten haben die PLU1 zu tragen',
            'Kosten hat die FEM1 zu tragen',
            'Kosten hat der MAS1 zu tragen',
            'PLU1 haben die Kosten zu tragen',
            'FEM1 hat die Kosten zu tragen',
            'MAS1 hat die Kosten zu tragen',
            'PLU1 tragen die Kosten',
            'FEM1 trägt die Kosten',
            'MAS1 trägt die Kosten',
            'Kosten tragen die PLU1',
            'Kosten trägt die FEM1',
            'Kosten trägt der MAS1',
            'Kosten werden den PLU2 auferlegt',
            'Kosten werden der FEM2 auferlegt',
            'Kosten werden dem MAS2 auferlegt',      
            'PLU2 werden die Kosten auferlegt',
            'FEM2 werden die Kosten auferlegt',
            'MAS2 werden die Kosten auferlegt']

dict1 = {'MAS1': ['Angeklagte','Beklagte','Antragsgegner'],
         'FEM1': ['Angeklagte','Beklagte','Antragsgegnerin','beklagte Partei'] ,
         'PLU1': ['Angeklagten','Beklagten','Antragsgegner'],
         'MAS2': ['Angeklagten','Beklagten','Antragsgegner'] ,
         'FEM2': ['Angeklagten','Beklagten','Antragsgegnerin','beklagten Partei'] ,
         'PLU2': ['Angeklagten','Beklagten','Antragsgegnern'],
        }

text_cl1 = ['werden verurteilt',
            'wird verurteilt']

for t in tmp_cl1:
    for j,(k,v) in enumerate(dict1.items()): 
        if t.find(k)>-1:
            for i,marker in enumerate(v):
                text_cl1 += [t.replace(k,marker)] 


text = text_cl0[:]
for t in text:
    if t.find('Kosten')>-1:
        text_cl0 += [t.replace('Kosten','Gesamtkosten')] 
           
            
text = text_cl1[:]
for t in text:
    if t.find('Kosten')>-1:
        text_cl1 += [t.replace('Kosten','Gesamtkosten')] 

to_remove = ['außergerichtlichen','gerichtlichen','und','notwendigen','gesamten','weiteren',
             'des Berufungsverfahrens','des Verfahrens','der Berufung','des Rechtsstreits', 
             'als Gesamtschuldner','als Gesamtschuldnerin','als Gesamtschuldnerinnen', 
             'seines Rechtsmittels','ihres Rechtsmittels','des Rechtsmittels']   

# all strings to lower case

for i,t in enumerate(to_remove):
    to_remove[i] = t.lower()    
        
for i,t in enumerate(text_cl0):
    text_cl0[i] = t.lower()    
    
for i,t in enumerate(text_cl1):
    text_cl1[i] = t.lower()     

text_cl0 = np.unique(text_cl0).tolist()
text_cl1 = np.unique(text_cl1).tolist()

# logical indices selecting certain elements of text_cl0 and text_cl1 
# ('dismissed','convicted','costs to plaintiff','costs to defendant')
id0a = [t in ['zurückgewiesen','abgewiesen'] for t in text_cl0]
id0b = [t not in ['zurückgewiesen','abgewiesen'] for t in text_cl0]
id1a = [t in ['werden verurteilt','wird verurteilt'] for t in text_cl1]
id1b = [t not in ['werden verurteilt','wird verurteilt'] for t in text_cl1]

In [ ]:
# create data dictionary from html files
#--------------------------------------------------------------------

# function converting html to text 
def get_text(i):
    with open(order[i],'r',encoding='utf-8') as f:
        html = f.read() 
        text= BeautifulSoup(html).get_text("\n",strip=True)   # for readability keep newline
    
    return text

# data dictionary
dd =  {'state': [],
       'court': [],
       'date': [],
       'decision': [],
       'offense': [],
       'dismissed': [],          # word appearance 'zurückgewiesen','abgewiesen'
       'convicted': [],          # word appearance 'werden verurteilt','wird verurteilt'
       'costs to plaintiff': [], # 'Kosten an klagende Partei'
       'costs to defendant': [], # 'Kosten an beklagte Partein'
      }

markers = ['\nGericht:','\nEntscheidungsdatum:','\nTenor\n','\nTatbestand\n','\nGründe\n','\nEntscheidungsgründe\n','\nGründe:','\nEntscheidungsgründe:','\nPermalink']
markersL = markers[:]
for i,t in enumerate(markers):
    markersL[i] = t.lower()  

k0 = markers.index('\nTenor\n')
k1 = markers.index('\nTatbestand\n')
k2 = markers.index('\nGericht:')
k3 = markers.index('\nEntscheidungsdatum:')
    
m = np.zeros([len(order),len(markers)])
m0 = np.zeros([len(order),len(text_cl0)])
m1 = np.zeros([len(order),len(text_cl1)])

for i in tqdm(range(len(order)), desc ="Read court orders"): 

    # load court order
    Text = get_text(i)
    text = Text.lower()
    
    # find location of markers within the court order
    for j, marker in enumerate(markers):
        m[i,j] = Text.find(marker)


    #  dismiss markers with colon (these don't occur anyway)
    if text.find('Tenor:')>-1:
        print('Skip order %g.' % (i))
        continue
        
    if text.find('Tatbestand:')>-1:
        print('Skip order %g.' % (i))
        continue

    n0 = int(m[i,k0])  # location of 'tenor'
    n1 = int(m[i,k1])  # location of 'Tatbestand'
    n2 = int(m[i,k2])  # location of 'Gericht:'
    n3 = int(m[i,k3])  # location of 'Entscheidungsdatum:'

    # check if 'tenor'and 'Tatbestand' are available
    if (n0>0)&(n1>0):
        # ni = int( np.min( m[i, m[i,:] > n0 ] ))   # Text from 'Tenor' to any next marker
        ni = int( np.min( np.concatenate( ([len(Text)],m[i, m[i,:] > n0 ]) )) ) # Text from 'Tenor' to any next marker
        nj = int( np.min( np.concatenate( ([len(Text)],m[i, m[i,:] > n1 ]) )) ) # Text from 'Tatbestand' to any next marker or end

        if (ni <= n0)|(nj <= n1):
            print('Check problem with order %g.' % (i))
            print(order[i])
        if (n0>n1):
            print('Check flipped Tenor and Tatbestand in order %g.' % (i))
            print(order[i])
        
        tenor = text[n0:ni]    
            
        # remove distracting fill words 
        for j, marker in enumerate(to_remove):
            tenor = tenor.replace(marker+' ','')
         
        for j, marker in enumerate(text_cl0):
            m0[i,j] = tenor.find(marker)
        
        for j, marker in enumerate(text_cl1):
            m1[i,j] = tenor.find(marker)

        # add data to dictionary
        date = Text[n3+21:n3+31] if n3>-1 else ''
        court = Text[n2+10:n3] if (n2>-1)&(n3>-1) else ''
        Tenor = Text[n0+7:ni]
        Tatbestand = Text[n1+12:nj]
            
        b0a = int( (m0[i,id0a]>-1).any() )
        b0b = int( (m0[i,id0b]>-1).any() )
        b1a = int( (m1[i,id1a]>-1).any() )
        b1b = int( (m1[i,id1b]>-1).any() )

        if not Tenor:
            print('Check empty Tenor in order %g.' % (i))
            continue
            
        if not Tatbestand:
            print('Check empty Tatbestand in order %g.' % (i))
            continue
            
        dd['state'].append(states[order_st[i]])
        dd['court'].append(court)
        dd['date'].append(date)
        dd['decision'].append(Tenor)               # Tenor (not in lower case)
        dd['offense'].append(Tatbestand)           # Tatbestand (not in lower case)
        dd['dismissed'].append(b0a)                # word appearance 'zurückgewiesen','abgewiesen'
        dd['convicted'].append(b1a)                # word appearance 'werden verurteilt','wird verurteilt'
        dd['costs to plaintiff'].append(b0b)       # 'Kosten an klagende Partei'
        dd['costs to defendant'].append(b1b)       # 'Kosten an beklagte Partein'


In [ ]:
# save dataset to json
savefilename1 = 'data_court_decisions.json'
df = pd.DataFrame(dd)
df.to_json(savefilename1,orient='records')
print(savefilename1)

# save dataset to jsonl
json_lines_dataset = load_dataset('json',data_files=savefilename1) # see if data is in the right json format to be loaded
savefilename2 = 'german_court_decisions'
json_lines_dataset.save_to_disk(savefilename2)
print(savefilename2)
